In [57]:
import re
import json
import sqlite3
import threading
import numpy as np
import requests as r
from csv import writer
from bs4 import BeautifulSoup as bs

In [58]:
search_url = " https://www.google.com/search?q="
user_agent = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36'
}

In [59]:
def validate(name):
    for i in name:
        if i not in 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890_' :
            j = name.index(i)
            name = name[:j:]+name[j+1::]
    return name

In [60]:
# Getting search result urls and filter to get only the working and relevent links

def search(item):
    print("searching for", item)
    search_result = r.get(search_url+item)

    if search_result.status_code != 200:
        print("ERROR! Searching Failed")
        quit()

    soup = bs(search_result.content, 'html.parser')
    search_result = soup.find("div", {"id": "main"})
    links = search_result.find_all('a')

    urls = []
    print("Filtering for relevent URLs")
    for link in links:
        if link['href'].startswith("/url?q=https://www."):
        
            url = link['href'].split("/url?q=")[1].split('&')[0]
            page = r.get(url, headers=user_agent)
        
            if page.status_code == 200:
                title = ''
                try:
                    title = bs(page.text, 'html.parser').title.text
                    if set(title.split()) & set(["Lifelong", "LLPCM05", "Trimmer"]):
                        urls.append(url)
                    else:
                        print("1m - ", title)
                except Exception as e:
                    print("1", title, e, link['href'])  

    return urls

In [61]:
#csv
def dump_to_csv(rows):
    np.savetxt("GFG.csv", rows, delimiter =", ", fmt ='% s', encoding = 'utf-8')

def csv_dump(reviews):
    print("creating csv dump")
    header = ['id', 'website', 'title', 'name', 'date', 'rating', 'r_text', 'address']
    i = 1
    with open("reviews.csv", 'w', encoding = 'utf-8') as f:
        write = writer(f)
        write.writerow(header)
        for row in reviews:
            r = (i,) + row
            write.writerow(r)
            i+=1


In [62]:

class amazon:
    def __init__(self) -> None:
        self.reviews: list
        
    def get_reviews(self, url):
        soup = bs(r.get(url, headers=user_agent).text, "html.parser")
        reviews = soup.find("div", {"id":"cm-cr-dp-review-list"})
        review_data = reviews.find_all("div", {"class":"a-section review aok-relative"})

        reviews = []
        for review in review_data:
            name = review.find("span", {"class": "a-profile-name"}).text.strip()
            rating = review.find('a', {"class": "a-link-normal"}).text.strip()
            title = review.find('a', {"data-hook":"review-title"}).text.strip()
            date = review.find('span', {"data-hook":"review-date"}).text.strip().split("on")[1]
            r_text = review.find("div", {"data-hook":"review-collapsed"}).text.strip()
            address = "India"
            reviews.append(("amazon", title, name, date, rating, r_text, address))
        self.reviews =  reviews

class flipkart:
    def __init__(self) -> None:
        self.reviews: list

    def get_reviews(self, url):
        soup = bs(r.get(url, headers=user_agent).text, "html.parser")
        reviews_data = soup.find_all("div", {"class": "col _2wzgFH"})

        reviews = []
        for review in reviews_data:
            name = review.find("p", {"class":"_2V5EHH"}).text.strip()
            rating = review.find("div", {"class":"_1BLPMq"}).text.strip()
            title = review.find('p', {"class":"_2-N8zT"}).text.strip()
            date = review.findAll('p', {"class":"_2sc7ZR"})[1].text.strip()
            r_text = review.find("div", {"class":"t-ZTKy"}).text.strip()
            address = review.find("p", {"class":"_2mcZGG"}).text.strip().split(", ")[1]
            reviews.append(("flipkart", title, name, date, rating, r_text, address))
        self.reviews = reviews

class snapdeal:
    def __init__(self) -> None:
        self.reviews: list

    def get_reviews(self, url):
        soup = re.compile("\"bucketLabelNames.*?,").findall(r.get(url, headers=user_agent).text)[0]
        page = r.post("https://www.snapdeal.com/acors/web/getReviewsAndRatings/v2", headers={
                "referer": url,
                "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36"
            }, json={
                "productId": url.split('/').pop(),
                "pageUrl": url,
                "bucketLabelNames":["appliances-personal1care1appliances-trimmers"]
                }
        ).text

        soup = bs(page, 'html.parser')
        review_data = soup.find("div", {"class":"commentreview"}).find_all("div", {"class": "text"})

        reviews = []
        for review in review_data:
            name_date = review.find("div", {"class":"_reviewUserName"})
            name = name_date["title"]
            date = name_date.text.strip().split("on")[1].strip()
            rating = len(review.find_all('i', {"class":"active"}))
            title = review.find("div", {"class": "head"}).text.strip()
            r_text = review.find('p').text.strip()
            address = ''
            reviews.append(("snapdeal", title, name, date, rating, r_text, address))
        self.reviews = reviews

class lifelongindiaonline:
    def __init__(self) -> None:
        self.reviews: list
        
    def get_reviews(self, url):

        soup = bs(r.get(url, headers=user_agent).text, "html.parser")
        product_id = json.loads(soup.find("script", {"id":"__st"}).text.split('=')[1][:-1])['rid']
        page = r.post(f"https://www.lifelongindiaonline.com/apps/aliexpress_reviews?product_id={product_id}&customer_id=", 
            headers={
                "referer": url,
                "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36"
            }
        ).text

        soup = bs(page, "html.parser")
        review_data = soup.find("div", {"id":"wrapper"}).find_all("div", {"class":"large-12 cell"})

        reviews = []
        for review in review_data:
            try:
                d = {}
                name = review.find("span", {"class":"areviews_user_name"}).text.strip()
                date = review.find('p', {"class":"comment_time comment_time_theme2"}).text.strip()
                rating = len(review.find("span", {"class":"main_container_review_stars"}).find_all('i', {"class":"fas fa-star"}))
                title = ''
                address = ''
                r_text= review.find("div", {"class":"comment_container"}).text.strip()
                reviews.append(("lifelongindiaonline", title, name, date, rating, r_text, address))
            except:
                pass
        self.reviews =  reviews

class jiomart:
    def __init__(self) -> None:
        self.reviews = []
    def get_reviews(self, url):
        pass
class croma:
    def __init__(self) -> None:
        self.reviews = []
    def get_reviews(self, url):
        pass
class smartprice:
    def __init__(self) -> None:
        self.reviews = []
    def get_reviews(self, url):
        pass
class reliancedigital:
    def __init__(self) -> None:
        self.reviews = []
    def get_reviews(self, url):
        pass
class pricehunt:
    def __init__(self) -> None:
        self.reviews = []
    def get_reviews(self, url):
        pass
class smartprix:
    def __init__(self) -> None:
        self.reviews = []
    def get_reviews(self, url):
        pass

In [63]:
# database
def create_databse():
    con = sqlite3.connect('database.db')
    cur = con.cursor()
    cur.execute("CREATE TABLE if not exists reviewdb(id integer PRIMARY KEY AUTOINCREMENT, website varchar(15) NOT NULL, title varchar(50), name varchar(25), date varchar(20), rating integer, review_text varchar(100), address varchar(20))")
    con.commit()
    con.close()
    print("database created")

def insert(reviews):
    print("Inserting reviews to database")
    con = sqlite3.connect('database.db')
    cur = con.cursor()
    print("inserting in database")
    cur.executemany("INSERT INTO reviewdb(website, title, name, date, rating, review_text, address) VALUES(?,?,?,?,?,?,?)", reviews)
    con.commit()
    con.close()
    return "reviews inserted into database"

def fetch():
    con = sqlite3.connect('database.db')
    cur = con.cursor()
    cur.execute("SELECT * FROM reviewdb")
    messages = cur.fetchall()
    con.close()
    return messages

def show_reviews():
    messages = fetch()
    for message in messages:
        print(message)

def database_dump():
    print("Creating database dump")
    con = sqlite3.connect('database.db')
    with open("database_dump.sql", 'w', encoding='utf-8') as f:
        f.write("\n".join(con.iterdump()))


In [64]:
def get_all_reviews(urls):
    threads_list = []
    website_objects = []

    for url in urls:
        website = validate(url.split('/')[2].split('.')[1])
        print(f"Fetching from - {website}, {url}")
        website_instance = eval(f"{website}()")
        website_objects.append(website_instance)
        t = threading.Thread(target = website_instance.get_reviews, args =(url,))
        t.start()
        threads_list.append(t)

    for t in threads_list:
            t.join()
    for t in threads_list:
        while t.is_alive():
            continue

    reviews = []
    for webobject in website_objects:
        if len(webobject.reviews):
            reviews.extend(webobject.reviews)
    
    return reviews


In [65]:
# save to database and csv and create dump
def main():
    item = "Lifelong LLPCM05 Beard Trimmer for Men"
    urls = search(item)
    reviews = get_all_reviews(urls)
    create_databse()
    insert(reviews)
    database_dump()
    csv_dump(reviews)
    print("COMPLETE !!!")


In [66]:
main()

searching for Lifelong LLPCM05 Beard Trimmer for Men
Filtering for relevent URLs
1  'NoneType' object has no attribute 'text' /url?q=https://www.tatacliq.com/lifelong-llpcm05-cordless-rechargeable-beard-trimmer-black/p-mp000000005426190&sa=U&ved=2ahUKEwiXz9z1nqP6AhVIuJUCHenyANwQFnoECAQQAg&usg=AOvVaw3R6ehkRbbZxiJNL5qbsWhu
Fetching from - amazon, https://www.amazon.in/Lifelong-LLPCM05-Beard-Trimmer-Cordless/dp/B07QGW4C1S
Fetching from - amazon, https://www.amazon.in/Lifelong-LLPCM05-Cordless-Trimmer-Warranty/dp/B084XK89XH
Fetching from - lifelongindiaonline, https://www.lifelongindiaonline.com/products/lifelong-llpcm05-beard-trimmeblack
Fetching from - flipkart, https://www.flipkart.com/lifelong-llpcm05-trimmer-45-min-runtime-8-length-settings/p/itm59598a15aafc8
Fetching from - snapdeal, https://www.snapdeal.com/product/lifelong-llpcm05-beard-trimmer-black-/675130634919
Fetching from - reliancedigital, https://www.reliancedigital.in/lifelong-llpcm05-men-s-beard-trimmer/p/491903023
Fetchi